In [118]:
!pip install -r requirements.txt

In [119]:
import pandas as pd
import numpy as np
import http.client
import requests
import pandas as pd
import json
import pprint
import seaborn as sns
import matplotlib.pyplot as plt

In [128]:
#Creating Nutritional Guidelines Dataframe
diet_req = pd.read_csv("dietary_req.csv").drop(['Source'],axis=1)
#Dividing by 3 to reflect meal
diet_req = diet_req.set_index('Nutrition')
dietreq_meal=diet_req.div(3)
dietreq_meal.iloc[len(dietreq_meal)-1,:] = dietreq_meal.iloc[len(dietreq_meal)-1,:] * -1

In [121]:
#Deliverable A: Creating Population Diet Requirement Function
groups = ['M 19-30','F 19-30']
def create_pop(group,activity):
    pop_req = dietreq_meal[group]
    if activity:
        pop_req.loc[0,group]= pop_req.iloc[0,1:3] + 300
    return pop_req
group = create_pop(groups, True).drop(0,axis=0)
group

/tmp/ipykernel_234/1045228780.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_req.loc[0,group]= pop_req.iloc[0,1:3] + 300


,M 19-30,F 19-30
Nutrition,,
Energy,800.000000,666.666667
Protein,18.666667,15.333333
"Fiber, total dietary",11.200000,9.333333
"Folate, DFE",133.333333,133.333333
"Calcium, Ca",333.333333,333.333333
"Carbohydrate, by difference",43.333333,43.333333
"Iron, Fe",2.666667,6.000000
"Magnesium, Mg",133.333333,103.333333
Niacin,5.333333,4.666667


In [122]:
#cleaning price + nutrition information database
price_nutrition = pd.read_csv('PriceData.csv')
price_nutrition=price_nutrition.set_index("Ingredients").drop(['Meal Name','Size','ID','fdcId','Brand'],axis=1)
price_nutrition=price_nutrition.drop('Carrots')
Prices = price_nutrition['Price'].astype(float)

In [142]:
# Deliverable A: Creating a function to find
from  scipy.optimize import linprog as lp
import warnings

def solve_subsistence_problem2(FoodNutrients,Prices,diet_min,max_weight=None,tol=1e-6):
    p = Prices.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = list(set(p.index.tolist()).intersection(FoodNutrients.columns.tolist()))
    p = p[use]
 
    Aall = FoodNutrients[p.index].fillna(0)
    Amin = Aall.loc[diet_min.index]
    A = Amin
    b= diet_min
    
    if max_weight is not None:
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
   
    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        result.diet = pd.Series(result.x,index=p.index)*np.nan  
    return result
    
result_M = solve_subsistence_problem2(price_nutrition.T,Prices,group['M 19-30'])
result_F = solve_subsistence_problem2(price_nutrition.T,Prices,group['F 19-30'])

In [143]:
#Printing our returned result for male population
print("Cost of diet for %s is $%4.2f per day.\n" % (['M 19-30'],result_M.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for ['M 19-30'] is $6.72 per day.


Diet (in 100s of grams or milliliters):
Ingredients
White Beans              1.693954
Pepper Bell Red Whole    0.459392
Parsley                  0.972841
Jalapeno Peppers         1.830053
Lemon Juice              0.221229
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                           800.000000      800.000000
Protein                           18.666667       18.666667
Fiber, total dietary              14.235794       11.200000
Folate, DFE                      345.248875      133.333333
Calcium, Ca                      609.126178      333.333333
Carbohydrate, by difference      146.515752       43.333333
Iron, Fe                          12.095585        2.666667
Magnesium, Mg                    197.516618      133.333333
Niacin                             5.333333        5.333333

In [144]:
#Printing our returned result for female population
print("Cost of diet for %s is $%4.2f per day.\n" % (['F 19-30'],result_F.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for ['F 19-30'] is $5.88 per day.


Diet (in 100s of grams or milliliters):
Ingredients
White Beans              1.693954
Pepper Bell Red Whole    0.459392
Parsley                  0.972841
Jalapeno Peppers         1.830053
Lemon Juice              0.221229
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                           800.000000      800.000000
Protein                           18.666667       18.666667
Fiber, total dietary              14.235794       11.200000
Folate, DFE                      345.248875      133.333333
Calcium, Ca                      609.126178      333.333333
Carbohydrate, by difference      146.515752       43.333333
Iron, Fe                          12.095585        2.666667
Magnesium, Mg                    197.516618      133.333333
Niacin                             5.333333        5.333333